# Title: BnSummarizer(Bangla Automatic Text Summarizer)  
# Student Registration No: 2015331542

# Problem:
 BnSummarizer is tool which summarizes bangla articles by extracting the most important sentences.

 # Solution Steps:
    1. The input text will be tokenized into sentences
    2. Calculating vector representation (word embeddings) for each and every sentence
    3. Distance between sentence vectors calculated storing them in a matrix
    4. Convert distance between sentences into graph
    5. In the graph sentences will be vertices and distance scores will be edges
    6. We will find the sentences will most frequency
    7. The output will be top 4 sentences
  
    - I will be using python3 and python packages 

# Code:
   - Every part of the code will be explained with a markdown cell. 

In [1]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt')
import re
from gensim.models import Word2Vec
import pyemd
import networkx as nx

unable to import 'smart_open.gcs', disabling that module


A method called `bnSent_tokenizer` is developed below to split bangla text into tokenized sentences as built in sent_tokenization method doesnt work correctly with Bangla.   
    - takes bangla text as arguement
    - returns a list of tokenized bangla sentences.

In [2]:
def bnSent_tokenizer(text):
    sentences = []
    sentence = ""
    for letter in text:
        if letter=='।':
            sentences.append(sentence) 
            sentence = ""
        else:
            sentence=sentence+letter
    return sentences

A method called `bnWord_tokenizer` is developed below to split bangla sentences into tokenized words
    - takes bangla text as arguement
    - returns a list of tokenized bangla sentences.

In [3]:
def bnWord_tokenizer(sentences):
    final_txt =[]
    for sentence in sentences:
        tokenized_sentence = sentence.split(" ")
        #print(tokenized_sentence)
        final_txt.append(tokenized_sentence)
    return final_txt


A method `train_model` develop to train word2vec model using out original text.
       - This method takes a list of list containing tokenized words.
       - returns word vectors

In [4]:
def train_model(final_txt):
    model = Word2Vec(final_txt, size=200, window=5, min_count=1)
    word_vectors = model.wv
    return word_vectors

We will create a method `distance_matrix` to creat a matrix of the word vector distance between sentences.
     - This method takes our trained model and list of list containing tokenized words as input
     - returns a distance matrix of square dimention (a numpy ndarray)

In [5]:
def distance_matrix(word_vectors,final_txt):
    dis_mat = np.zeros([len(final_txt), len(final_txt)])
    for i in range(len(final_txt)):
        for j in range(len(final_txt)):
            dis_mat[i][j]=word_vectors.wmdistance(final_txt[i],final_txt[j])
    return dis_mat


Converting the distance matrix into a graph using `senGraph` method . The nodes of this graph will represent the sentences and the edges will represent the distance scores between the sentences. 
    - takes the distance matrixs as arguement 
    - returns sentence rankings.

In [6]:
def senGraph(dis_mat):
    nx_graph = nx.from_numpy_array(dis_mat)
    scores = nx.pagerank(nx_graph)
    return scores

Method `extractSum` to get the summary. 
       - takes tokenzied sentences and sentence graph aka scores as input
       - returns summary as text

In [7]:
def extractSum(sentences,scores):
    sorted_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    summary = "" 
    for i in range(6):
        try:
            summary=summary+sorted_sentences[i][1]+"। "
        except IndexError:
            break
    return summary

Finally the `BnSummarizer` method to find the summary.
    - takes a bangla text as input
    - return a summary string as output

In [8]:
def BnSummarizer(bnString):
    sentences = bnSent_tokenizer(bnString)
    final_txt = bnWord_tokenizer(sentences)
    word_vectors = train_model(final_txt)
    dis_mat = distance_matrix(word_vectors,final_txt)
    scores = senGraph(dis_mat)
    summary = extractSum(sentences,scores)
    return summary

# Input Sample: 
The first part of [Oporichita By Rabindranath Tagore](https://bn.wikisource.org/wiki/%E0%A6%97%E0%A6%B2%E0%A7%8D%E0%A6%AA%E0%A6%97%E0%A7%81%E0%A6%9A%E0%A7%8D%E0%A6%9B/%E0%A6%85%E0%A6%AA%E0%A6%B0%E0%A6%BF%E0%A6%9A%E0%A6%BF%E0%A6%A4%E0%A6%BE) is used here as input. 

In [9]:
origin_text = input("Enter the Bangla text to summarize: ") 

Enter the Bangla text to summarize: আজ আমার বয়স সাতাশ মাত্র। এ জীবনটা না দৈর্ঘ্যের হিসাবে বড়াে, না গুণের হিসাবে। তবু ইহার একটু বিশেষ মূল্য আছে। ইহা সেই ফুলের মতাে যাহার বুকের উপরে ভ্রমর আসিয়া বসিয়াছিল, এবং সেই পদক্ষেপের ইতিহাস তাহার জীবনের মাঝখানে ফলের মতাে গুটি ধরিয়া উঠিয়াছে।   সেই ইতিহাসটুকু আকারে ছােটো, তাহাকে ছােটো করিয়াই লিখিব। ছােটোকে যাঁহারা সামান্য বলিয়া ভুল করেন না তাঁহারা ইহার রস বুঝিবেন।   কলেজে যতগুলাে পরীক্ষা পাস করিবার সব আমি চুকাইয়াছি। ছেলেবেলায় আমার সুন্দর চেহারা লইয়া পণ্ডিতমশায় আমাকে শিমুল ফুল ও মাকাল ফলের সহিত তুলনা করিয়া বিদ্রুপ করিবার সুযােগ পাইয়াছিলেন। ইহাতে তখন বড়ো লজ্জা পাইতাম; কিন্তু বয়স হইয়া এ কথা ভাবিয়াছি, যদি জন্মান্তর থাকে তবে আমার মুখে সরূপ এবং পণ্ডিতমশায়দের মুখে বিদ্রুপ আবার যেন এমনি করিয়াই প্রকাশ পায়।   আমার পিতা এক কালে গরিব ছিলেন। ওকালতি করিয়া তিনি প্রচুর টাকা রােজগার করিয়াছেন, ভােগ করিবার সময় নিমেষমাত্রও পান নাই। মৃত্যুতে তিনি যে হাঁফ ছাড়িলেন সেই তাঁর প্রথম অবকাশ।   আমার তখন বয়স অল্প। মার হাতেই আমি মানুষ। মা গরিবের ঘরের মেয়ে;

# Output Sample:

In [14]:
print("The Bangla Summary goes here: \n"+BnSummarizer(origin_text)) 

The Bangla Summary goes here: 
আজ আমার বয়স সাতাশ মাত্র।  সামনে যত দূর পর্যন্ত দৃষ্টি চলে ছুটি ধূ ধূ করিতেছে; পরীক্ষা নাই, উমেদারি নাই, চাকরি নাই; নিজের বিষয় দেখিবার চিন্তাও নাই, শিক্ষাও নাই, ইচ্ছাও নাই—থাকিবার মধ্যে ভিতরে আছেন মা এবং বাহিরে আছেন মামা।  ইহাতে তখন বড়ো লজ্জা পাইতাম; কিন্তু বয়স হইয়া এ কথা ভাবিয়াছি, যদি জন্মান্তর থাকে তবে আমার মুখে সরূপ এবং পণ্ডিতমশায়দের মুখে বিদ্রুপ আবার যেন এমনি করিয়াই প্রকাশ পায়।  ইহা সেই ফুলের মতাে যাহার বুকের উপরে ভ্রমর আসিয়া বসিয়াছিল, এবং সেই পদক্ষেপের ইতিহাস তাহার জীবনের মাঝখানে ফলের মতাে গুটি ধরিয়া উঠিয়াছে।  একে তাে বরের হাট মহার্ঘ, তাহার পরে ধনুক-ভাঙা পণ, কাজেই বাপ কেবলই সবুর করিতেছেন- কিন্তু মেয়ের বয়স সবুর করিতেছে না।  যাহাকে শোষণ করা চলিবে অথচ বাড়িতে আসিলে গুড়গুড়ির পরিবর্তে বাঁধা হুঁকায় তামাক দিলে যাহার নালিশ খাটিবে না। 


# Github: 
https://github.com/tajrian/BanglaTxtSum/blob/master/BnSummarizer.ipynb

 - Typically it returns summary in 6 sentencs, it can be changed using the `extractSum` method
 - original text which has less or equal 6 sentences,results in returning the same original text


# Submitted by:
    Tajrian Binta Jahid 
    Reg No: 2015331542
    Department of Computer Science and Engineering
    Sylhet Engineering College
